In [6]:
# 첫번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "1"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    



from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 두번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "2"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )
test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 세번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "3"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 네번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "4"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 다섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "5"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 여섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
numb = "6"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=128,callbacks=[early_stopping]
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)




100%|█████████████████████████████████████████████████████████████████████████████| 2280/2280 [00:06<00:00, 366.89it/s]


Epoch 1/100
57/57 [==============================] - 26s 443ms/step - loss: 0.6750 - acc: 0.5929 - val_loss: 0.6390 - val_acc: 0.6595
Epoch 2/100
57/57 [==============================] - 24s 426ms/step - loss: 0.5984 - acc: 0.6989 - val_loss: 0.6095 - val_acc: 0.6842
Epoch 3/100
57/57 [==============================] - 25s 438ms/step - loss: 0.5490 - acc: 0.7357 - val_loss: 0.6080 - val_acc: 0.6880
Epoch 4/100
57/57 [==============================] - 24s 418ms/step - loss: 0.5063 - acc: 0.7584 - val_loss: 0.6228 - val_acc: 0.6826
Epoch 5/100
57/57 [==============================] - 25s 434ms/step - loss: 0.4777 - acc: 0.7811 - val_loss: 0.6443 - val_acc: 0.6787
Epoch 6/100
57/57 [==============================] - 25s 429ms/step - loss: 1.4266 - acc: 0.6007 - val_loss: 0.9315 - val_acc: 0.5576
Epoch 7/100
57/57 [==============================] - 25s 443ms/step - loss: 0.5203 - acc: 0.7485 - val_loss: 0.6365 - val_acc: 0.6809
Epoch 8/100
57/57 [==============================] - 26s 456ms

100%|█████████████████████████████████████████████████████████████████████████████| 1880/1880 [00:05<00:00, 345.49it/s]


Epoch 1/100
47/47 [==============================] - 22s 437ms/step - loss: 0.6802 - acc: 0.5833 - val_loss: 0.6609 - val_acc: 0.6263
Epoch 2/100
47/47 [==============================] - 21s 449ms/step - loss: 0.6132 - acc: 0.6810 - val_loss: 0.6306 - val_acc: 0.6516
Epoch 3/100
47/47 [==============================] - 21s 450ms/step - loss: 0.5588 - acc: 0.7269 - val_loss: 0.6407 - val_acc: 0.6543
Epoch 4/100
47/47 [==============================] - 21s 440ms/step - loss: 0.5149 - acc: 0.7566 - val_loss: 0.6556 - val_acc: 0.6549
Epoch 5/100
47/47 [==============================] - 22s 471ms/step - loss: 0.4796 - acc: 0.7824 - val_loss: 0.6765 - val_acc: 0.6536
Epoch 6/100
47/47 [==============================] - 23s 491ms/step - loss: 0.4422 - acc: 0.8090 - val_loss: 0.6931 - val_acc: 0.6423
Epoch 7/100
47/47 [==============================] - 21s 439ms/step - loss: 0.4134 - acc: 0.8208 - val_loss: 0.7704 - val_acc: 0.6277
Epoch 8/100
47/47 [==============================] - 21s 441ms

100%|█████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:04<00:00, 302.54it/s]


...

Epoch 1/100
27/27 [==============================] - 14s 462ms/step - loss: 0.6881 - acc: 0.5648 - val_loss: 0.6756 - val_acc: 0.6655
Epoch 2/100
27/27 [==============================] - 12s 460ms/step - loss: 0.6375 - acc: 0.6774 - val_loss: 0.6206 - val_acc: 0.6921
Epoch 3/100
27/27 [==============================] - 12s 459ms/step - loss: 0.5685 - acc: 0.7329 - val_loss: 0.6122 - val_acc: 0.6829
Epoch 4/100
27/27 [==============================] - 12s 463ms/step - loss: 0.5395 - acc: 0.7451 - val_loss: 0.6210 - val_acc: 0.6759
Epoch 5/100
27/27 [==============================] - 12s 460ms/step - loss: 0.4708 - acc: 0.7943 - val_loss: 0.6307 - val_acc: 0.6678
Epoch 6/100
27/27 [==============================] - 13s 468ms/step - loss: 0.4300 - acc: 0.8148 - val_loss: 0.6477 - val_acc: 0.6562
Epoch 7/100
27/27 [==============================] - 13s 470ms/step - loss: 0.4292 - acc: 0.8171 - val_loss: 0.6639 - val_acc: 0.6424
Epoch 8/100
27/27 [==============================] - 12s 457ms

100%|███████████████████████████████████████████████████████████████████████████████| 680/680 [00:02<00:00, 255.27it/s]


Epoch 1/100
17/17 [==============================] - 9s 479ms/step - loss: 0.6905 - acc: 0.5207 - val_loss: 0.6856 - val_acc: 0.6066
Epoch 2/100
17/17 [==============================] - 7s 438ms/step - loss: 0.6680 - acc: 0.6374 - val_loss: 0.6661 - val_acc: 0.6213
Epoch 3/100
17/17 [==============================] - 8s 464ms/step - loss: 0.6183 - acc: 0.6889 - val_loss: 0.6641 - val_acc: 0.5809
Epoch 4/100
17/17 [==============================] - 8s 447ms/step - loss: 0.6709 - acc: 0.7017 - val_loss: 0.6491 - val_acc: 0.6287
Epoch 5/100
17/17 [==============================] - 8s 456ms/step - loss: 0.5446 - acc: 0.7679 - val_loss: 0.6384 - val_acc: 0.6213
Epoch 6/100
17/17 [==============================] - 8s 455ms/step - loss: 0.5304 - acc: 0.7753 - val_loss: 1.6272 - val_acc: 0.5184
Epoch 7/100
17/17 [==============================] - 7s 441ms/step - loss: 0.6666 - acc: 0.7335 - val_loss: 0.6497 - val_acc: 0.6250
Epoch 8/100
17/17 [==============================] - 7s 430ms/step - 

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 384.59it/s]


Epoch 1/100
7/7 [==============================] - 5s 530ms/step - loss: 0.6921 - acc: 0.5045 - val_loss: 0.6910 - val_acc: 0.5804
Epoch 2/100
7/7 [==============================] - 3s 452ms/step - loss: 0.6828 - acc: 0.6808 - val_loss: 0.6859 - val_acc: 0.5670
Epoch 3/100
7/7 [==============================] - 3s 459ms/step - loss: 0.6631 - acc: 0.6696 - val_loss: 0.6778 - val_acc: 0.5670
Epoch 4/100
7/7 [==============================] - 3s 484ms/step - loss: 0.6332 - acc: 0.7098 - val_loss: 0.6700 - val_acc: 0.5670
Epoch 5/100
7/7 [==============================] - 3s 485ms/step - loss: 0.5945 - acc: 0.7377 - val_loss: 0.6687 - val_acc: 0.5714
Epoch 6/100
7/7 [==============================] - 3s 450ms/step - loss: 0.5493 - acc: 0.7902 - val_loss: 0.6594 - val_acc: 0.5804
Epoch 7/100
7/7 [==============================] - 3s 450ms/step - loss: 0.4939 - acc: 0.8203 - val_loss: 0.6646 - val_acc: 0.5893
Epoch 8/100
7/7 [==============================] - 3s 447ms/step - loss: 0.5132 - a

In [3]:
# 첫번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "1"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    



from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 두번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "2"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 세번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "3"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 네번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "4"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 다섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "5"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 여섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
numb = "6"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)




100%|█████████████████████████████████████████████████████████████████████████████| 2280/2280 [00:05<00:00, 382.38it/s]


Epoch 1/10
57/57 [==============================] - 24s 398ms/step - loss: 0.6870 - acc: 0.5504 - val_loss: 0.6825 - val_acc: 0.5499
Epoch 2/10
57/57 [==============================] - 22s 388ms/step - loss: 0.6664 - acc: 0.5958 - val_loss: 0.6792 - val_acc: 0.5691
Epoch 3/10
57/57 [==============================] - 22s 389ms/step - loss: 0.6242 - acc: 0.6623 - val_loss: 0.7010 - val_acc: 0.5559
Epoch 4/10
57/57 [==============================] - 22s 389ms/step - loss: 0.5714 - acc: 0.7067 - val_loss: 0.7408 - val_acc: 0.5488
Epoch 5/10
57/57 [==============================] - 22s 388ms/step - loss: 0.5253 - acc: 0.7370 - val_loss: 0.7874 - val_acc: 0.5609
Epoch 6/10
57/57 [==============================] - 22s 390ms/step - loss: 0.4847 - acc: 0.7611 - val_loss: 0.8249 - val_acc: 0.5543
Epoch 7/10
57/57 [==============================] - 22s 389ms/step - loss: 0.5238 - acc: 0.7734 - val_loss: 0.8856 - val_acc: 0.5461
Epoch 8/10
57/57 [==============================] - 22s 390ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1880/1880 [00:05<00:00, 369.22it/s]


Epoch 1/10
47/47 [==============================] - 20s 399ms/step - loss: 0.6896 - acc: 0.5389 - val_loss: 0.6851 - val_acc: 0.5559
Epoch 2/10
47/47 [==============================] - 18s 393ms/step - loss: 0.6677 - acc: 0.5919 - val_loss: 0.6813 - val_acc: 0.5725
Epoch 3/10
47/47 [==============================] - 18s 393ms/step - loss: 0.6234 - acc: 0.6569 - val_loss: 0.6949 - val_acc: 0.5638
Epoch 4/10
47/47 [==============================] - 18s 394ms/step - loss: 0.5727 - acc: 0.6950 - val_loss: 0.7205 - val_acc: 0.5532
Epoch 5/10
47/47 [==============================] - 18s 392ms/step - loss: 0.5235 - acc: 0.7357 - val_loss: 0.7656 - val_acc: 0.5618
Epoch 6/10
47/47 [==============================] - 18s 393ms/step - loss: 0.4808 - acc: 0.7650 - val_loss: 0.8444 - val_acc: 0.5505
Epoch 7/10
47/47 [==============================] - 18s 391ms/step - loss: 0.4406 - acc: 0.7892 - val_loss: 0.8841 - val_acc: 0.5392
Epoch 8/10
47/47 [==============================] - 18s 392ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:03<00:00, 391.55it/s]


...

Epoch 1/10
27/27 [==============================] - 12s 408ms/step - loss: 0.6914 - acc: 0.5292 - val_loss: 0.6869 - val_acc: 0.5671
Epoch 2/10
27/27 [==============================] - 11s 398ms/step - loss: 0.6796 - acc: 0.5810 - val_loss: 0.6777 - val_acc: 0.5706
Epoch 3/10
27/27 [==============================] - 11s 395ms/step - loss: 0.6530 - acc: 0.6299 - val_loss: 0.6656 - val_acc: 0.6030
Epoch 4/10
27/27 [==============================] - 11s 393ms/step - loss: 0.6034 - acc: 0.6913 - val_loss: 0.6775 - val_acc: 0.5914
Epoch 5/10
27/27 [==============================] - 11s 397ms/step - loss: 0.5464 - acc: 0.7433 - val_loss: 0.7008 - val_acc: 0.5706
Epoch 6/10
27/27 [==============================] - 11s 397ms/step - loss: 0.4865 - acc: 0.7810 - val_loss: 0.7310 - val_acc: 0.5567
Epoch 7/10
27/27 [==============================] - 11s 395ms/step - loss: 0.4795 - acc: 0.7836 - val_loss: 0.7712 - val_acc: 0.5637
Epoch 8/10
27/27 [==============================] - 11s 395ms/step - 

100%|███████████████████████████████████████████████████████████████████████████████| 680/680 [00:01<00:00, 403.10it/s]


Epoch 1/10
17/17 [==============================] - 8s 420ms/step - loss: 0.6934 - acc: 0.5005 - val_loss: 0.6920 - val_acc: 0.5202
Epoch 2/10
17/17 [==============================] - 7s 395ms/step - loss: 0.6862 - acc: 0.6135 - val_loss: 0.6896 - val_acc: 0.5037
Epoch 3/10
17/17 [==============================] - 7s 397ms/step - loss: 0.6682 - acc: 0.6158 - val_loss: 0.6866 - val_acc: 0.5331
Epoch 4/10
17/17 [==============================] - 7s 400ms/step - loss: 0.6316 - acc: 0.6590 - val_loss: 0.6882 - val_acc: 0.5441
Epoch 5/10
17/17 [==============================] - 7s 396ms/step - loss: 0.6445 - acc: 0.7206 - val_loss: 0.6965 - val_acc: 0.5423
Epoch 6/10
17/17 [==============================] - 7s 397ms/step - loss: 0.6608 - acc: 0.7335 - val_loss: 0.7093 - val_acc: 0.5368
Epoch 7/10
17/17 [==============================] - 7s 398ms/step - loss: 0.5366 - acc: 0.7753 - val_loss: 1.6482 - val_acc: 0.5000
Epoch 8/10
17/17 [==============================] - 7s 395ms/step - loss: 0.

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 362.64it/s]


Epoch 1/10
7/7 [==============================] - 4s 458ms/step - loss: 0.6936 - acc: 0.4955 - val_loss: 0.6920 - val_acc: 0.5536
Epoch 2/10
7/7 [==============================] - 3s 403ms/step - loss: 0.6889 - acc: 0.6295 - val_loss: 0.6890 - val_acc: 0.5491
Epoch 3/10
7/7 [==============================] - 3s 395ms/step - loss: 0.6774 - acc: 0.6596 - val_loss: 0.6819 - val_acc: 0.5312
Epoch 4/10
7/7 [==============================] - 3s 392ms/step - loss: 0.6710 - acc: 0.6105 - val_loss: 0.6784 - val_acc: 0.5580
Epoch 5/10
7/7 [==============================] - 3s 399ms/step - loss: 0.6433 - acc: 0.6696 - val_loss: 0.6758 - val_acc: 0.5670
Epoch 6/10
7/7 [==============================] - 3s 403ms/step - loss: 0.6191 - acc: 0.7210 - val_loss: 0.6742 - val_acc: 0.5625
Epoch 7/10
7/7 [==============================] - 3s 408ms/step - loss: 0.5880 - acc: 0.7388 - val_loss: 0.6721 - val_acc: 0.5893
Epoch 8/10
7/7 [==============================] - 3s 407ms/step - loss: 0.5588 - acc: 0.76

In [4]:
# 첫번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "1"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    



from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 두번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "2"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 세번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "3"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 네번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "4"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 다섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "5"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)
# 여섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "6"
filename1 = './spy/definition'+name+'_spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/definition'+name+'_spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/definition'+name+'_spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/definition'+name+'_spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)




100%|█████████████████████████████████████████████████████████████████████████████| 2280/2280 [00:05<00:00, 385.83it/s]


Epoch 1/10
57/57 [==============================] - 24s 400ms/step - loss: 0.6926 - acc: 0.5100 - val_loss: 0.6919 - val_acc: 0.5280
Epoch 2/10
57/57 [==============================] - 22s 392ms/step - loss: 0.6765 - acc: 0.6010 - val_loss: 0.6888 - val_acc: 0.5554
Epoch 3/10
57/57 [==============================] - 23s 397ms/step - loss: 0.6279 - acc: 0.6553 - val_loss: 0.7108 - val_acc: 0.5471
Epoch 4/10
57/57 [==============================] - 22s 391ms/step - loss: 0.5794 - acc: 0.6970 - val_loss: 0.7470 - val_acc: 0.5340
Epoch 5/10
57/57 [==============================] - 22s 394ms/step - loss: 0.5397 - acc: 0.7296 - val_loss: 0.7681 - val_acc: 0.5318
Epoch 6/10
57/57 [==============================] - 22s 391ms/step - loss: 0.5196 - acc: 0.7419 - val_loss: 0.8208 - val_acc: 0.5230
Epoch 7/10
57/57 [==============================] - 22s 389ms/step - loss: 0.4632 - acc: 0.7722 - val_loss: 0.8883 - val_acc: 0.5214
Epoch 8/10
57/57 [==============================] - 22s 393ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1880/1880 [00:05<00:00, 360.71it/s]


Epoch 1/10
47/47 [==============================] - 20s 399ms/step - loss: 0.6930 - acc: 0.5073 - val_loss: 0.6921 - val_acc: 0.5386
Epoch 2/10
47/47 [==============================] - 18s 391ms/step - loss: 0.6813 - acc: 0.6001 - val_loss: 0.6885 - val_acc: 0.5519
Epoch 3/10
47/47 [==============================] - 18s 391ms/step - loss: 0.6344 - acc: 0.6508 - val_loss: 0.7125 - val_acc: 0.5299
Epoch 4/10
47/47 [==============================] - 18s 391ms/step - loss: 0.5808 - acc: 0.7006 - val_loss: 0.7342 - val_acc: 0.5346
Epoch 5/10
47/47 [==============================] - 18s 390ms/step - loss: 0.5372 - acc: 0.7354 - val_loss: 1.0894 - val_acc: 0.5120
Epoch 6/10
47/47 [==============================] - 18s 392ms/step - loss: 0.5410 - acc: 0.7379 - val_loss: 0.7693 - val_acc: 0.5246
Epoch 7/10
47/47 [==============================] - 18s 391ms/step - loss: 0.4637 - acc: 0.7862 - val_loss: 0.8330 - val_acc: 0.5379
Epoch 8/10
47/47 [==============================] - 18s 391ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:03<00:00, 396.16it/s]


...

Epoch 1/10
27/27 [==============================] - 12s 404ms/step - loss: 0.6930 - acc: 0.5078 - val_loss: 0.6928 - val_acc: 0.5104
Epoch 2/10
27/27 [==============================] - 11s 391ms/step - loss: 0.6870 - acc: 0.6218 - val_loss: 0.6909 - val_acc: 0.5382
Epoch 3/10
27/27 [==============================] - 11s 393ms/step - loss: 0.6619 - acc: 0.6476 - val_loss: 0.6881 - val_acc: 0.5532
Epoch 4/10
27/27 [==============================] - 11s 393ms/step - loss: 0.6168 - acc: 0.6982 - val_loss: 0.6932 - val_acc: 0.5625
Epoch 5/10
27/27 [==============================] - 10s 390ms/step - loss: 0.5612 - acc: 0.7321 - val_loss: 0.7098 - val_acc: 0.5671
Epoch 6/10
27/27 [==============================] - 11s 392ms/step - loss: 0.5183 - acc: 0.7549 - val_loss: 0.7314 - val_acc: 0.5648
Epoch 7/10
27/27 [==============================] - 11s 392ms/step - loss: 0.4705 - acc: 0.7847 - val_loss: 0.7537 - val_acc: 0.5521
Epoch 8/10
27/27 [==============================] - 11s 390ms/step - 

100%|███████████████████████████████████████████████████████████████████████████████| 680/680 [00:01<00:00, 403.12it/s]


Epoch 1/10
17/17 [==============================] - 8s 415ms/step - loss: 0.6932 - acc: 0.5055 - val_loss: 0.6933 - val_acc: 0.5037
Epoch 2/10
17/17 [==============================] - 7s 391ms/step - loss: 0.6879 - acc: 0.6218 - val_loss: 0.6930 - val_acc: 0.5129
Epoch 3/10
17/17 [==============================] - 7s 391ms/step - loss: 0.6716 - acc: 0.6673 - val_loss: 0.6933 - val_acc: 0.5165
Epoch 4/10
17/17 [==============================] - 7s 390ms/step - loss: 0.6237 - acc: 0.7040 - val_loss: 0.7202 - val_acc: 0.5184
Epoch 5/10
17/17 [==============================] - 7s 392ms/step - loss: 0.5550 - acc: 0.7463 - val_loss: 0.7200 - val_acc: 0.5294
Epoch 6/10
17/17 [==============================] - 7s 395ms/step - loss: 0.4945 - acc: 0.7904 - val_loss: 0.7504 - val_acc: 0.5257
Epoch 7/10
17/17 [==============================] - 7s 398ms/step - loss: 0.4481 - acc: 0.8162 - val_loss: 0.7486 - val_acc: 0.5515
Epoch 8/10
17/17 [==============================] - 7s 400ms/step - loss: 0.

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 357.70it/s]


Epoch 1/10
7/7 [==============================] - 4s 444ms/step - loss: 0.6935 - acc: 0.4754 - val_loss: 0.6928 - val_acc: 0.5268
Epoch 2/10
7/7 [==============================] - 3s 400ms/step - loss: 0.6899 - acc: 0.6406 - val_loss: 0.6925 - val_acc: 0.5446
Epoch 3/10
7/7 [==============================] - 3s 401ms/step - loss: 0.6829 - acc: 0.7243 - val_loss: 0.6924 - val_acc: 0.5268
Epoch 4/10
7/7 [==============================] - 3s 404ms/step - loss: 0.6667 - acc: 0.7779 - val_loss: 0.6933 - val_acc: 0.5223
Epoch 5/10
7/7 [==============================] - 3s 407ms/step - loss: 0.6388 - acc: 0.7210 - val_loss: 0.6944 - val_acc: 0.5580
Epoch 6/10
7/7 [==============================] - 3s 403ms/step - loss: 0.5869 - acc: 0.8248 - val_loss: 0.7037 - val_acc: 0.5268
Epoch 7/10
7/7 [==============================] - 3s 405ms/step - loss: 0.5994 - acc: 0.8080 - val_loss: 0.7071 - val_acc: 0.5491
Epoch 8/10
7/7 [==============================] - 3s 400ms/step - loss: 0.4858 - acc: 0.88

In [5]:
# 첫번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "1"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    



from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)


# 두번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "2"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)


# 세번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "3"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)


# 네번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "4"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

print("acc : ",test_acc)

# 다섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "5"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)


# 여섯번째

from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
numb = "6"
filename1 = './spy/spynorth_scaling_trust'+numb+'.txt'
filename2 = './spy/spynorth_scaling_untrust'+numb+'.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()


# test set


with codecs.open('./spy/spynorth_test_t'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./spy/spynorth_test_ut'+numb+'.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()


    


from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1
lines_ut = lines2
test_t = test1
test_ut = test2


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

test_loss , test_acc = model.evaluate(input_test, test_labels)
print("acc : ",test_acc)






100%|█████████████████████████████████████████████████████████████████████████████| 2280/2280 [00:05<00:00, 385.85it/s]


Epoch 1/10
57/57 [==============================] - 24s 400ms/step - loss: 0.6916 - acc: 0.5193 - val_loss: 0.6888 - val_acc: 0.5296
Epoch 2/10
57/57 [==============================] - 22s 395ms/step - loss: 0.6701 - acc: 0.6003 - val_loss: 0.6758 - val_acc: 0.5784
Epoch 3/10
57/57 [==============================] - 23s 397ms/step - loss: 0.6260 - acc: 0.6627 - val_loss: 0.6842 - val_acc: 0.5784
Epoch 4/10
57/57 [==============================] - 22s 393ms/step - loss: 0.5788 - acc: 0.7057 - val_loss: 0.7102 - val_acc: 0.5680
Epoch 5/10
57/57 [==============================] - 22s 394ms/step - loss: 1.1325 - acc: 0.6081 - val_loss: 1.0738 - val_acc: 0.5455
Epoch 6/10
57/57 [==============================] - 22s 393ms/step - loss: 0.8505 - acc: 0.6109 - val_loss: 0.7701 - val_acc: 0.5417
Epoch 7/10
57/57 [==============================] - 22s 393ms/step - loss: 0.7861 - acc: 0.5910 - val_loss: 0.8003 - val_acc: 0.5312
Epoch 8/10
57/57 [==============================] - 22s 394ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1880/1880 [00:04<00:00, 384.76it/s]


Epoch 1/10
47/47 [==============================] - 20s 401ms/step - loss: 0.6927 - acc: 0.5170 - val_loss: 0.6912 - val_acc: 0.5465
Epoch 2/10
47/47 [==============================] - 18s 391ms/step - loss: 0.6767 - acc: 0.6084 - val_loss: 0.6842 - val_acc: 0.5552
Epoch 3/10
47/47 [==============================] - 18s 393ms/step - loss: 0.6337 - acc: 0.6556 - val_loss: 0.6913 - val_acc: 0.5705
Epoch 4/10
47/47 [==============================] - 18s 394ms/step - loss: 0.5922 - acc: 0.6941 - val_loss: 0.7040 - val_acc: 0.5592
Epoch 5/10
47/47 [==============================] - 18s 392ms/step - loss: 0.5609 - acc: 0.7156 - val_loss: 0.7148 - val_acc: 0.5698
Epoch 6/10
47/47 [==============================] - 18s 393ms/step - loss: 0.5193 - acc: 0.7595 - val_loss: 0.7440 - val_acc: 0.5565
Epoch 7/10
47/47 [==============================] - 18s 393ms/step - loss: 0.5453 - acc: 0.7483 - val_loss: 0.7669 - val_acc: 0.5539
Epoch 8/10
47/47 [==============================] - 18s 391ms/step - 

100%|█████████████████████████████████████████████████████████████████████████████| 1480/1480 [00:03<00:00, 376.69it/s]


...

Epoch 1/10
27/27 [==============================] - 12s 402ms/step - loss: 0.6924 - acc: 0.5208 - val_loss: 0.6911 - val_acc: 0.5440
Epoch 2/10
27/27 [==============================] - 10s 389ms/step - loss: 0.6749 - acc: 0.6207 - val_loss: 0.6865 - val_acc: 0.5532
Epoch 3/10
27/27 [==============================] - 10s 389ms/step - loss: 0.6291 - acc: 0.6832 - val_loss: 0.6979 - val_acc: 0.5556
Epoch 4/10
27/27 [==============================] - 11s 392ms/step - loss: 0.5744 - acc: 0.7216 - val_loss: 0.7218 - val_acc: 0.5556
Epoch 5/10
27/27 [==============================] - 10s 388ms/step - loss: 0.5373 - acc: 0.7381 - val_loss: 0.7429 - val_acc: 0.5440
Epoch 6/10
27/27 [==============================] - 10s 389ms/step - loss: 0.4975 - acc: 0.7668 - val_loss: 0.7524 - val_acc: 0.5498
Epoch 7/10
27/27 [==============================] - 10s 386ms/step - loss: 0.4597 - acc: 0.8108 - val_loss: 0.7831 - val_acc: 0.5405
Epoch 8/10
27/27 [==============================] - 10s 389ms/step - 

100%|███████████████████████████████████████████████████████████████████████████████| 680/680 [00:01<00:00, 397.09it/s]


Epoch 1/10
17/17 [==============================] - 8s 419ms/step - loss: 0.6932 - acc: 0.5000 - val_loss: 0.6921 - val_acc: 0.5294
Epoch 2/10
17/17 [==============================] - 7s 397ms/step - loss: 0.6860 - acc: 0.6287 - val_loss: 0.6892 - val_acc: 0.5680
Epoch 3/10
17/17 [==============================] - 7s 393ms/step - loss: 0.6614 - acc: 0.6889 - val_loss: 0.6822 - val_acc: 0.5643
Epoch 4/10
17/17 [==============================] - 7s 399ms/step - loss: 0.6050 - acc: 0.7270 - val_loss: 0.7001 - val_acc: 0.5515
Epoch 5/10
17/17 [==============================] - 7s 396ms/step - loss: 0.5402 - acc: 0.7753 - val_loss: 0.7040 - val_acc: 0.5735
Epoch 6/10
17/17 [==============================] - 7s 397ms/step - loss: 0.4834 - acc: 0.8001 - val_loss: 0.7144 - val_acc: 0.5680
Epoch 7/10
17/17 [==============================] - 7s 399ms/step - loss: 0.4298 - acc: 0.8346 - val_loss: 0.7984 - val_acc: 0.5386
Epoch 8/10
17/17 [==============================] - 7s 393ms/step - loss: 0.

100%|███████████████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 360.67it/s]


Epoch 1/10
7/7 [==============================] - 4s 448ms/step - loss: 0.6931 - acc: 0.4944 - val_loss: 0.6922 - val_acc: 0.5670
Epoch 2/10
7/7 [==============================] - 3s 392ms/step - loss: 0.6875 - acc: 0.6507 - val_loss: 0.6906 - val_acc: 0.5223
Epoch 3/10
7/7 [==============================] - 3s 396ms/step - loss: 0.6746 - acc: 0.7188 - val_loss: 0.6888 - val_acc: 0.5491
Epoch 4/10
7/7 [==============================] - 3s 402ms/step - loss: 0.6513 - acc: 0.7221 - val_loss: 0.6842 - val_acc: 0.5536
Epoch 5/10
7/7 [==============================] - 3s 401ms/step - loss: 0.6063 - acc: 0.7355 - val_loss: 0.6831 - val_acc: 0.5625
Epoch 6/10
7/7 [==============================] - 3s 404ms/step - loss: 0.5452 - acc: 0.8013 - val_loss: 0.6912 - val_acc: 0.5625
Epoch 7/10
7/7 [==============================] - 3s 400ms/step - loss: 0.4905 - acc: 0.8884 - val_loss: 0.6976 - val_acc: 0.5357
Epoch 8/10
7/7 [==============================] - 3s 407ms/step - loss: 0.6583 - acc: 0.73